# Import and Load Data

In [70]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
from IPython.display import display, Math

## modules I am still trying to learn how to use
import numexpr
import bottleneck

##
pd.set_option('display.float_format', lambda x: '%.3e' % x)
np.set_printoptions(formatter={'all':lambda x: '%.3e'% x})

## Functions

In [2]:
## function to extract fits and hdf data
def load_true():
    """This is a function that takes fits and hdf data and return them to pandas dataframe"""

    # detect = fits.open('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits')[-1]
    phot   = fits.open('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits')[1]
    id_balrog = fits.open('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits')

    #load true catalogs in pandas
    # detectp_table=Table.read('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits').to_pandas()
    photdf=Table.read('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits').to_pandas()
    id_balrogdf = Table.read('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits').to_pandas()
    
    photdf.drop_duplicates(['bal_id'],inplace=True)
    
    photdf.set_index('bal_id',inplace=True,verify_integrity=True)

    return(photdf)


def load_mcal():
    #all objects riz
#     mcal0= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-v1.2.h5','r')['catalog']['unsheared'] #with neighbor
#     mcal1= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-v1.2.h5','r')['catalog']['unsheared'] #no neighbor
    ##only matched balrog object riz
    mcal2=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal3=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    #only matched balrog object griz
    mcal4=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal5=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    
    mcal_list=[mcal2,mcal3,mcal4,mcal5]
    return (mcal_list)

In [3]:
## function to convert 
def get_df_list(mcal_list):
    mcal_df_list =[]
    for catalog in mcal_list:
        local_df=pd.DataFrame()
        for key in catalog.keys():
            local_df[key]=np.array(catalog[key][:]).byteswap().newbyteorder()
        if 'bal_id' in local_df.keys():
            local_df.set_index('bal_id',inplace=True,verify_integrity=True)
        mcal_df_list.append(local_df)
    return(mcal_df_list)

In [4]:
photdf=load_true()

In [5]:
mcal_df_list=get_df_list(load_mcal())

## List of dtypes in mcal

In [6]:
mcal_df_list[2].dtypes

R11                    >f8
R12                    >f8
R21                    >f8
R22                    >f8
T                  float64
T_err              float64
coadd_object_id      int64
covmat_0_1         float64
covmat_1_1         float64
covmat_2_2         float64
dec                float64
e_1                float64
e_2                float64
flags                int32
flux_err_g         float64
flux_err_i         float64
flux_err_r         float64
flux_err_z         float64
flux_g             float64
flux_i             float64
flux_r             float64
flux_z             float64
mask_frac          float64
mcal_psf_T         float64
mcal_psf_e1        float64
mcal_psf_e2        float64
nimage_tot_g         int32
nimage_tot_i         int32
nimage_tot_r         int32
nimage_tot_z         int32
nimage_use_g         int32
nimage_use_i         int32
nimage_use_r         int32
nimage_use_z         int32
psf_T              float64
psf_e1             float64
psf_e2             float64
r

## List of dtypes in phot

In [7]:
photdf.dtypes

true_id                    int32
true_number                int32
true_flags                 int32
true_obj_flags             int32
true_cm_mof_flags          int32
true_ra                  float64
true_dec                 float64
true_cm_T                float64
true_cm_T_err            float64
true_cm_s2n_r            float64
true_cm_flags              int32
true_cm_fracdev          float64
true_cm_TdByTe           float32
true_in_VHS_footprint      int32
true_mag_auto_det        float32
true_det_number            int32
true_tilename             object
meas_id                    int64
meas_number                int32
meas_ra                  float64
meas_dec                 float64
meas_fofid                 int64
meas_flags                 int32
meas_time_last_fit       float64
meas_box_size              int16
meas_obj_flags             int32
meas_mask_frac           float64
meas_psfrec_T            float64
meas_cm_flags              int32
meas_cm_T                float64
          

## sample of mcal data

In [52]:
mcal_df_list[3].head()

,R11,R12,R21,R22,T,T_err,coadd_object_id,covmat_0_1,covmat_1_1,covmat_2_2,...,nimage_use_g,nimage_use_i,nimage_use_r,nimage_use_z,psf_T,psf_e1,psf_e2,ra,size_ratio,snr
bal_id,,,,,,,,,,,,,,,,,,,,,
100002002078,7.292e-01,-9.135e-02,-1.480e-01,8.401e-01,6.269e-01,2.868e-02,1240,-2.624e-06,5.007e-04,4.912e-04,...,2,3,3,2,6.476e-01,4.030e-03,1.082e-02,7.408e-01,9.681e-01,3.844e+01
100002041238,8.345e-01,1.588e-06,2.283e-03,8.298e-01,1.169e+00,1.456e-03,29227,-1.264e-10,3.840e-07,3.837e-07,...,5,3,4,6,5.310e-01,-9.231e-03,2.081e-02,3.970e-01,2.202e+00,2.464e+03
100002042490,1.808e-01,-5.235e-01,6.396e-03,8.761e-01,5.687e-01,6.062e-02,23493,3.710e-05,1.247e-03,1.079e-03,...,2,4,2,4,4.382e-01,-7.386e-03,2.014e-02,6.565e-01,1.298e+00,2.096e+01
100002042496,8.733e-01,-4.114e-02,-3.950e-02,1.066e+00,5.280e+00,7.866e-03,11311,4.360e-10,2.613e-07,2.597e-07,...,2,3,3,3,5.333e-01,-2.135e-02,2.057e-02,6.602e-01,9.901e+00,8.942e+02
100003037062,2.542e-01,1.004e+00,3.476e-01,1.686e+00,4.365e-01,6.705e-02,33370,-2.213e-04,3.516e-03,3.329e-03,...,3,3,4,5,5.003e-01,-6.316e-03,-1.852e-02,8.960e-01,8.724e-01,1.237e+01


# Data manipulation

## Constants

In [9]:
SNR_CUT = 10
SIZE_RATIO_CUT = 0.5
T_CUT = 1000

## Filter by SNR T and size_ratio

In [10]:
def filter_by_snr():
    for i,catalog in enumerate(mcal_df_list):
        mcal_df_list[i]=catalog[(catalog['snr']>SNR_CUT) & (catalog['T']<T_CUT) & (catalog['size_ratio']>SIZE_RATIO_CUT)]

In [11]:
filter_by_snr()

In [16]:
for mcal in mcal_df_list:
    print(len(mcal))

4365353
4338058
4382905
4382602


## Intersects paris of catalogs by bal_id. We only want objects existing in true catalog, MOF and woMOF  

In [12]:
index_2_3=mcal_df_list[0].index.intersection(mcal_df_list[1].index)
index_4_5=mcal_df_list[2].index.intersection(mcal_df_list[3].index)

index_2_3_true=index_2_3.intersection(photdf.index).sort_values()
index_4_5_true=index_4_5.intersection(photdf.index).sort_values()

In [13]:
mcal_df_list[0]=mcal_df_list[0].reindex(index_2_3_true);
mcal_df_list[1]=mcal_df_list[1].reindex(index_2_3_true);
mcal_df_list[2]=mcal_df_list[2].reindex(index_4_5_true);
mcal_df_list[3]=mcal_df_list[3].reindex(index_4_5_true);

In [28]:
true_df_list=[None]*4
for i in range(2):
    true_df_list[i]=photdf.reindex(index_2_3_true)
    true_df_list[i+2]=photdf.reindex(index_4_5_true)

In [33]:
for i,catalog in enumerate(mcal_df_list[0:4]):
    print("The lenth of mcal{} is {}.".format(i+2,len(catalog.index)))
#     print(catalog.T.isnull().any())

The lenth of mcal2 is 1098637.
The lenth of mcal3 is 1098637.
The lenth of mcal4 is 867808.
The lenth of mcal5 is 867808.


In [32]:
for i, catalog in enumerate(true_df_list):
    print ("The length of corresponding true catalog is {}.".format(len(catalog.index)))

The length of corresponding true catalog is 1098637.
The length of corresponding true catalog is 1098637.
The length of corresponding true catalog is 867808.
The length of corresponding true catalog is 867808.


# Reorganize array for easy calculation and display

## Get R'

We read $R$ matrix from the measured catalog. And we calculate $R'$ matrix with $$R_{ij}=\frac{<e_i>}{<g_j>}$$

In [44]:
R_list=[None]*4
R_prime_list=[None]*4
for i,catalog in enumerate(mcal_df_list):
    true_catalog=true_df_list[i]
    
    e1=catalog['e_1'].mean()
    e2=catalog['e_2'].mean()
    g1=true_catalog['true_cm_g_1'].mean()
    g2=true_catalog['true_cm_g_2'].mean()
    
    R_list[i]=[[catalog['R11'].mean(),catalog['R12'].mean()],[catalog['R21'].mean(),catalog['R22'].mean()]]
    R_prime_list[i]=[[e1/g1,e1/g2],[e2/g1,e2/g2]]

In [77]:
for i,R in enumerate(R_list):
    print ("For mcal {}".format(i+2))
    display(Math('R='))
    print(np.matrix(R))
    display(Math('R\prime='))
    print(np.matrix(R_prime_list[i]))
    print("\n")

For mcal 2


<IPython.core.display.Math object>

[[7.769e-01 -1.044e-05]
 [-1.955e-04 7.828e-01]]


<IPython.core.display.Math object>

[[8.732e-01 2.079e+01]
 [1.138e-01 2.710e+00]]


For mcal 3


...

[[9.397e-01 5.954e+00]
 [2.031e-01 1.287e+00]]


For mcal 5


<IPython.core.display.Math object>

[[7.916e-01 1.613e-03]
 [2.058e-03 7.959e-01]]


<IPython.core.display.Math object>

[[8.985e-01 5.693e+00]
 [1.886e-01 1.195e+00]]




# Getting dictionary arrays

In [ ]:
for i,R_array in enumerate(R_array_negative):
    e1_error=np.std(negative_filtered[i]['e_1'])/np.sqrt(float(len(negative_filtered[i]['e_1'])))
    e2_error=np.std(negative_filtered[i]['e_2'])/np.sqrt(float(len(negative_filtered[i]['e_2'])))
    
    g1_error=np.std(negative_true[i]['true_cm_g_1'])/np.sqrt(float(len(negative_true[i]['true_cm_g_1'])))
    g2_error=np.std(negative_true[i]['true_cm_g_2'])/np.sqrt(float(len(negative_true[i]['true_cm_g_2'])))
    
    e1_mean=np.mean(negative_filtered[i]['e_1'])
    e2_mean=np.mean(negative_filtered[i]['e_2'])
    
    g1_mean=np.mean(negative_true[i]['true_cm_g_1'])
    g2_mean=np.mean(negative_true[i]['true_cm_g_2'])
    
    R11_error=np.sqrt((1/g1_mean)**2 * e1_error**2 + (e1_mean/(g1_mean**2))**2*g1_error**2)
    R22_error=np.sqrt((1/g2_mean)**2 * e2_error**2 + (e2_mean/(g2_mean**2))**2*g2_error**2)
    
    R_11_mean=np.mean(negative_filtered[i]['R11'])
    R_22_mean=np.mean(negative_filtered[i]['R22'])
    R_11_std=np.std(negative_filtered[i]['R11'])
    R_22_std=np.std(negative_filtered[i]['R22'])

    
    R_11_read_error=R_11_std/np.sqrt(float(len(negative_filtered[i]['R11'])))
    R_22_read_error=R_11_std/np.sqrt(float(len(negative_filtered[i]['R22'])))

    
    
    g_prime_1=np.mean(negative_filtered[i]['e_1'])/R_11_mean
    g_prime_2=np.mean(negative_filtered[i]['e_2'])/R_22_mean
    
    g_prime_1_error=np.sqrt((1/R_11_mean)**2 * e1_error**2 + (e1_mean/R_11_mean**2)**2*R_11_read_error**2)
    g_prime_2_error=np.sqrt((1/R_22_mean)**2 * e2_error**2 + (e2_mean/R_22_mean**2)**2*R_22_read_error**2)
    print("For mcal{} with negative g, measured R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*row1))
    print("<e1>={:5.3e}, <e2>={:5.3e}, <g1>={:5.3e}, <g2>={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(*eg_array_negative[i][0],*eg_array_negative[i][1],real_R_array_negative[i][0][0],real_R_array_negative[i][1][1]))
    print("d_e1={:5.3e}, d_e2={:5.3e}".format(e1_error,e2_error))
    print("R11'_error={:5.3e} R22'_error={:5.3e}".format(R11_error,R22_error))
    print("\n")
    print ("g1'={:5.3e}, g2'={:5.3e},".format(g_prime_1,g_prime_2)+" g1'_error={:5.3e}, g2'_error={:5.3e}".format(g_prime_1_error,g_prime_2_error))
    print("\n")
    print("Theoretical R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*real_R_array_negative[i][j]))
    print ("R11'={:5.3e}, R22'={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(R_array_negative[i][0][0],R_array_negative[i][1][1],real_R_array_negative[i][0][0],real_R_array_negative[i][1][1]))
    print("-----------------------------------------------------------------")
    print("\n")


In [ ]:
for i,R_array in enumerate(R_array_positive):
    e1_error=np.std(positive_filtered[i]['e_1'])/np.sqrt(float(len(positive_filtered[i]['e_1'])))
    e2_error=np.std(positive_filtered[i]['e_2'])/np.sqrt(float(len(positive_filtered[i]['e_2'])))
    g1_error=np.std(positive_true[i]['true_cm_g_1'])/np.sqrt(float(len(positive_true[i]['true_cm_g_1'])))
    g2_error=np.std(positive_true[i]['true_cm_g_2'])/np.sqrt(float(len(positive_true[i]['true_cm_g_2'])))
    e1_mean=np.mean(positive_filtered[i]['e_1'])
    e2_mean=np.mean(positive_filtered[i]['e_2'])
    g1_mean=np.mean(positive_true[i]['true_cm_g_1'])
    g2_mean=np.mean(positive_true[i]['true_cm_g_2'])
    R11_error=np.sqrt((1/g1_mean)**2 * e1_error**2 + (e1_mean/(g1_mean**2))**2*g1_error**2)
    R22_error=np.sqrt((1/g2_mean)**2 * e2_error**2 + (e2_mean/(g2_mean**2))**2*g2_error**2)
    
    R_11_mean=np.mean(positive_filtered[i]['R11'])
    R_22_mean=np.mean(positive_filtered[i]['R22'])
    R_11_std=np.std(negative_filtered[i]['R11'])
    R_22_std=np.std(negative_filtered[i]['R22'])
    
    R_11_read_error=R_11_std/np.sqrt(float(len(positive_filtered[i]['R11'])))
    R_22_read_error=R_22_std/np.sqrt(float(len(positive_filtered[i]['R22'])))
    
    g_prime_1=np.mean(positive_filtered[i]['e_1'])/R_11_mean
    g_prime_2=np.mean(positive_filtered[i]['e_2'])/R_22_mean
    
    g_prime_1_error=np.sqrt((1/R_11_mean)**2 * e1_error**2 + (e1_mean/(R_11_mean**2))**2*R_11_read_error**2)
    g_prime_2_error=np.sqrt((1/R_22_mean)**2 * e2_error**2 + (e2_mean/(R_22_mean**2))**2*R_22_read_error**2)

    

    print("For mcal{} with positive g, measured R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*row1))
    print("<e1>={:5.3e}, <e2>={:5.3e}, <g1>={:5.3e}, <g2>={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(*eg_array_positive[i][0],*eg_array_positive[i][1],real_R_array_positive[i][0][0],real_R_array_positive[i][1][1]))
    print("d_e1={:5.3e}, d_e2={:5.3e}".format(e1_error,e2_error))
    print("R11'_error={:5.3e} R22'_error={:5.3e}".format(R11_error,R22_error))
    print("\n")
    print ("g1'={:5.3e}, g2'={:5.3e},".format(g_prime_1,g_prime_2)+" g1'_error={:5.3e}, g2'_error={:5.3e}".format(g_prime_1_error,g_prime_2_error))
    
    print("Theoretical R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*real_R_array_positive[i][j]))
    print ("R11'={:5.3e}, R22'={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(R_array_positive[i][0][0],R_array_positive[i][1][1],real_R_array_positive[i][0][0],real_R_array_positive[i][1][1]))
    print("-----------------------------------------------------------------")

In [ ]:
print ("For positive_g")
for i in [0,2]:
    print ("The difference beween R matrice in mcal{:5.3e} and mcal{:5.3e} is:".format(i+2,i+3))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_positive[i][j])-np.array(R_array_positive[i+1][j]))))

In [ ]:
print ("For negative g")
for i in [0,2]:
    print ("The difference beween R matrice in mcal{} and mcal{} is:".format(i+2,i+3))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_negative[i][j])-np.array(R_array_negative[i+1][j]))))

In [ ]:
print ("For positive g")
for i in range(4):
    print("For mcal{}, the diffrence between measured and theoretical R is".format(i+2))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_positive[i][j])-np.array(real_R_array_positive[i][j])))) 

In [ ]:
print ("For negative g")
for i in range(4):
    print("For mcal{}, the diffrence between measured and theoretical R is".format(i+2))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_negative[i][j])-np.array(real_R_array_negative[i][j])))) 

# Fitting e_i = m_i * g + c

In [ ]:
intersected_masks=[get_intersected_mask(true_dict_unique_sorted,catalog) for catalog in relavant_catalogs]

In [ ]:
intersected_masks_for_mcal=[mask[1] for mask in intersected_masks]
intersected_masks_for_true=[mask[0] for mask in intersected_masks]

In [ ]:
intersected_clipped_array=mask_catalog_arrays(relavant_catalogs,intersected_masks_for_mcal)

In [ ]:
four_true_dicts=[true_dict_unique_sorted for i in range(4)]

In [ ]:
intersected_clipped_true=mask_catalog_arrays(four_true_dicts,intersected_masks_for_true)

In [ ]:
def get_cm(true_dict,mcal_dict):
        y1=mcal_dict['e_1']
        x1=true_dict['true_cm_g_1']
        y2=mcal_dict['e_2']
        x2=true_dict['true_cm_g_2']
        m1,c1=np.polyfit(x1,y1,1)
        m2,c2=np.polyfit(x2,y2,1)
        return([[m1,c1],[m2,c2]])

In [ ]:
cm_array=[get_cm(intersected_clipped_true[i],intersected_clipped_array[i]) for i in range(4)]

In [ ]:
cm_array_positive=[get_cm(positive_true[i],positive_filtered[i]) for i in range(4)]

In [ ]:
cm_array_negative=[get_cm(negative_true[i],negative_filtered[i]) for i in range(4)]

In [ ]:
def print_cm(cm_array):
    for i,matrix in enumerate(cm_array):
        print("For mcal{}, m1={:10.3e},c1={:10.3e},m2={:10.3e},c2={:10.3e}".format((i+2),*matrix[0],*matrix[1]))

In [ ]:
print_cm(cm_array)

In [ ]:
print_cm(cm_array_positive)

In [ ]:
print_cm(cm_array_negative)

In [ ]:
x1_array=[true_dict['true_cm_g_1'] for true_dict in intersected_clipped_true]
x2_array=[true_dict['true_cm_g_2'] for true_dict in intersected_clipped_true]
y1_array=[mcal_dict['e_1'] for mcal_dict in intersected_clipped_array]
y2_array=[mcal_dict['e_2'] for mcal_dict in intersected_clipped_array]

In [ ]:
x3_array=[true_dict['true_cm_g_1'] for true_dict in positive_true]
x4_array=[true_dict['true_cm_g_2'] for true_dict in positive_true]
y3_array=[mcal_dict['e_1'] for mcal_dict in positive_filtered]
y4_array=[mcal_dict['e_2'] for mcal_dict in positive_filtered]

In [ ]:
x5_array=[true_dict['true_cm_g_1'] for true_dict in negative_true]
x6_array=[true_dict['true_cm_g_2'] for true_dict in negative_true]
y5_array=[mcal_dict['e_1'] for mcal_dict in negative_filtered]
y6_array=[mcal_dict['e_2'] for mcal_dict in negative_filtered]

In [ ]:
colors=(0,0,0)
area=0.01
def plot_scatter(x,y,cm):
    plt.scatter(x,y,s=area,c=colors,alpha=0.5)
    x_range=np.linspace(0,1)
    plt.plot(x_range,cm[0]*x_range+cm[1])

In [ ]:
for i in range(4):
    plt.figure()
    plot_scatter(x1_array[i],y1_array[i],cm_array[i][0])
    plt.title("mcal{} g_1 e_1".format(i+2))
    plt.savefig("scatter/mcal{}g1e1".format(i+2),dpi=300)
    plt.close()
    plt.figure()
    plot_scatter(x2_array[i],y2_array[i],cm_array[i][1])
    plt.title("mcal{} g_2 e_2".format(i+2))
    plt.savefig("scatter/mcal{}g2e2".format(i+2),dpi=300)
    plt.close()

In [ ]:
for i in range(4):
    plt.figure()
    plot_scatter(x3_array[i],y3_array[i],cm_array_positive[i][0])
    plt.title("mcal{} g_1 e_1_positive".format(i+2))
    plt.savefig("scatter/mcal{}g1e1_positive".format(i+2),dpi=300)
    plt.close()
    plt.figure()
    plot_scatter(x4_array[i],y4_array[i],cm_array[i][1])
    plt.title("mcal{} g_2 e_2_positive".format(i+2))
    plt.savefig("scatter/mcal{}g2e2_positive".format(i+2),dpi=300)
    plt.close()

In [ ]:
for i in range(4):
    plt.figure()
    plot_scatter(x3_array[i],y3_array[i],cm_array_positive[i][0])
    plt.title("mcal{} g_1 e_1_negative".format(i+2))
    plt.savefig("scatter/mcal{}g1e1_negative".format(i+2),dpi=300)
    plt.close()
    plt.figure()
    plot_scatter(x4_array[i],y4_array[i],cm_array[i][1])
    plt.title("mcal{} g_2 e_2_negative".format(i+2))
    plt.savefig("scatter/mcal{}g2e2_negative".format(i+2),dpi=300)
    plt.close()

In [ ]:
for i in [0,2]:
    print(i)

# healpix

In [ ]:
from astropy_healpix import HEALpix
from astropy.coordinate import icrs

